In [1]:
import pandas as pd 
import glob
import os 
import re

In [2]:
# 1) check bamfiles with fastq files
def checkFastq(task_file = "../../../fun/res/blca_task_found.csv",R1R2 = r"R1|R2"):
    import pandas as pd 
    import glob
    import os 
    import re
    task_not_found_fastq = []
    task_found_fastq = []
    task_found_fastq_more_than2 = []
    df_list = []
    # 1) get dataframe
    task_df = pd.read_csv(task_file)
    for each_dir in task_df["file_position"]:
        each_dir = str(each_dir).rstrip("/")
        # 1) get dirrectory
        each_name = os.path.basename(each_dir)
        # 2) check fastq 
        fq_found = glob.glob(each_dir + "/*gz")
        if len(fq_found) == 0:
            task_not_found_fastq.append(each_name)
        elif len(fq_found) == 2:
            task_found_fastq.append(each_name)
            df_list.append(
                pd.DataFrame({
                    "sample" : [each_name] * 2,
                    "file_position" :fq_found}))
        else:
            task_found_fastq_more_than2.append(each_name)
    print("How many samples not fastq?")
    print(len(task_not_found_fastq))
    print("How many samples with 2 fastq?")
    print(len(task_found_fastq))
    print("How many samples with more than 2 fastq?")
    print(len(task_found_fastq_more_than2))

    # get reads info 
    df  = pd.concat(df_list)
    df["reads"] =[re.search(R1R2,x)[0] for x in df["file_position"]]
    
    df = df.pivot(columns=["reads"],values=["file_position"],index="sample")
    df.columns = ["R1","R2"]
    return df 

df = checkFastq()

How many samples not fastq?
0
How many samples with 2 fastq?
132
How many samples with more than 2 fastq?
0


In [4]:
df.head()

,R1,R2
sample,,
L211206026-ZLZ,/usr/local/m3data/task_data/WS522-211225/L2112...,/usr/local/m3data/task_data/WS522-211225/L2112...
L211209801-ZLZ,/usr/local/m3data/task_data/WS522-211225/L2112...,/usr/local/m3data/task_data/WS522-211225/L2112...
L211209801-ZLZ-B,/usr/local/m3data/task_data/WS522-211225/L2112...,/usr/local/m3data/task_data/WS522-211225/L2112...
L220106801-ZLZ,/usr/local/m3data/task_data/WS035-220127/L2201...,/usr/local/m3data/task_data/WS035-220127/L2201...
L220106801-ZLZ-B,/usr/local/m3data/task_data/WS035-220127/L2201...,/usr/local/m3data/task_data/WS035-220127/L2201...


In [6]:
df.to_csv("blca_sample_meta.csv")